# LLM Zoomcamp - Module 1 Homework

This notebook contains the solutions for the Module 1 homework.

## Setup and Imports

First, let's install necessary libraries if you haven't already:
```bash
pip install requests openai elasticsearch python-dotenv tiktoken tqdm pandas
```
Ensure you have a `.env` file in the same directory as this notebook (or your OpenAI API key is set as an environment variable):
```env
OPENAI_API_KEY="your_openai_api_key_here"
```
And make sure your Elasticsearch instance is running (e.g., using Docker for version 8.17.0 as suggested by the homework context, though the specific version might vary slightly in your setup):
```bash
docker run -it --rm --name elasticsearch -p 9200:9200 -p 9300:9300 -e "discovery.type=single-node" -e "xpack.security.enabled=false" docker.elastic.co/elasticsearch/elasticsearch:8.17.0
```

In [2]:
# Common imports and setup for the notebook
import requests
import os
import json
from dotenv import load_dotenv
from openai import OpenAI
from elasticsearch import Elasticsearch
from tqdm.auto import tqdm
import tiktoken
import pandas as pd

# Load environment variables (for OpenAI API key)
load_dotenv()

# Initialize OpenAI client
# Ensure OPENAI_API_KEY is in your .env file or environment
try:
    client = OpenAI(api_key=os.environ.get("OPENAI_API_KEY"))
    if not os.environ.get("OPENAI_API_KEY"):
        print("Warning: OPENAI_API_KEY not found. OpenAI calls will fail.")
except Exception as e:
    print(f"Error initializing OpenAI client: {e}")

# Initialize Elasticsearch client
# Make sure your Elasticsearch instance is running
try:
    es_client = Elasticsearch('http://localhost:9200')
    if not es_client.ping(): # Check connection
        raise ConnectionError("Failed to connect to Elasticsearch at http://localhost:9200")
    print("Successfully connected to Elasticsearch.")
except ConnectionError as e:
    print(f"ConnectionError: {e}. Please ensure Elasticsearch is running and accessible.")
    # You might want to stop execution here or handle it gracefully
except Exception as e:
    print(f"An unexpected error occurred when connecting to Elasticsearch: {e}")

Successfully connected to Elasticsearch.


## Q1: Elasticsearch Version

**Goal:** Get the `version.build_hash` from your running Elasticsearch instance.

In [3]:
# Q1: Get Elasticsearch version.build_hash
build_hash_q1 = None
try:
    if es_client.ping(): # Ensure client is connected
        info = es_client.info()
        build_hash_q1 = info['version']['build_hash']
        print(f"Q1 Answer: Elasticsearch version.build_hash: {build_hash_q1}")
    else:
        print("Cannot get Elasticsearch info for Q1. Client not connected.")
except Exception as e:
    print(f"Error getting Elasticsearch info for Q1: {e}")
    print("Please ensure Elasticsearch (e.g., version 8.17.0) is running and accessible.")

Q1 Answer: Elasticsearch version.build_hash: 42f05b9372a9a4a470db3b52817899b99a76ee73


## Getting the FAQ Data

**Goal:** Fetch the FAQ data from the provided URL.

In [4]:
docs_url = 'https://github.com/DataTalksClub/llm-zoomcamp/blob/main/01-intro/documents.json?raw=true'
docs_response = requests.get(docs_url)
documents_raw = docs_response.json()

documents = []
for course_data in documents_raw:
    course_name = course_data['course']
    for doc in course_data['documents']:
        doc['course'] = course_name
        documents.append(doc)

print(f"Successfully loaded {len(documents)} documents.")
# print(documents[0]) # To inspect the first document

Successfully loaded 948 documents.


In [5]:
print(documents[0]) # To inspect the first document

{'text': "The purpose of this document is to capture frequently asked technical questions\nThe exact day and hour of the course will be 15th Jan 2024 at 17h00. The course will start with the first  “Office Hours'' live.1\nSubscribe to course public Google Calendar (it works from Desktop only).\nRegister before the course starts using this link.\nJoin the course Telegram channel with announcements.\nDon’t forget to register in DataTalks.Club's Slack and join the channel.", 'section': 'General course-related questions', 'question': 'Course - When will the course start?', 'course': 'data-engineering-zoomcamp'}


## Q2: Indexing Data

**Goal:** Index the fetched FAQ data. `course` field as `keyword`, others as `text`. Identify the function for adding data.

**Which function do you use for adding your data to elastic?**
* `insert`
* **`index`**
* `put`
* `add`

The correct function is `index`.

In [6]:
# Q2: Indexing data
index_name = "course-faq-homework-2025"

# Define index settings and mappings
index_settings = {
    "settings": {
        "number_of_shards": 1,
        "number_of_replicas": 0
    },
    "mappings": {
        "properties": {
            "course": {"type": "keyword"},
            "question": {"type": "text"},
            "section": {"type": "text"},
            "text": {"type": "text"}
        }
    }
}

try:
    if es_client.indices.exists(index=index_name):
        es_client.indices.delete(index=index_name)
        print(f"Index '{index_name}' deleted.")
    es_client.indices.create(index=index_name, body=index_settings)
    print(f"Index '{index_name}' created.")

    # Index the documents
    for doc in tqdm(documents):
        es_client.index(index=index_name, document=doc)
    print(f"Q2: Successfully indexed {len(documents)} documents into '{index_name}'.")
    print("The function used for adding data is 'es_client.index()'.")

except ConnectionError:
    print("Elasticsearch connection error for Q2. Cannot perform indexing.")
except Exception as e:
    print(f"An error occurred during indexing for Q2: {e}")

Index 'course-faq-homework-2025' deleted.
Index 'course-faq-homework-2025' created.


  0%|          | 0/948 [00:00<?, ?it/s]

Q2: Successfully indexed 948 documents into 'course-faq-homework-2025'.
The function used for adding data is 'es_client.index()'.


## Q3: Searching with Boost

**Goal:** Query "How do execute a command on a Kubernetes pod?". Use `question` (boost 4) and `text` fields. Type `best_fields`. Find the top score.

Options:
* 84.50
* 64.50
* 44.50
* 24.50

In [7]:
# Q3: Searching with boost
query_q3 = "How do execute a command on a Kubernetes pod?"
score_q3_answer = None

search_query_q3 = {
    "size": 1, 
    "query": {
        "multi_match": {
            "query": query_q3,
            "fields": ["question^4", "text"], 
            "type": "best_fields"
        }
    }
}

try:
    response_q3 = es_client.search(index=index_name, body=search_query_q3)
    if response_q3['hits']['hits']:
        top_hit_q3 = response_q3['hits']['hits'][0]
        score_q3_answer = top_hit_q3['_score']
        print(f"Q3 Answer: The score for the top ranking result is: {score_q3_answer}")
    else:
        print(f"No results found for the query in Q3: '{query_q3}'")
except ConnectionError:
    print("Elasticsearch connection error for Q3. Cannot perform search.")
except Exception as e:
    print(f"An error occurred during search Q3: {e}")

Q3 Answer: The score for the top ranking result is: 44.50556


## Q4: Filtered Search

**Goal:** Query "How do copy a file to a Docker container?". Filter for `machine-learning-zoomcamp`. Return 3 results. Identify the 3rd question.

Options:
* How do I debug a docker container?
* How do I copy files from a different folder into docker container’s working directory?
* How do Lambda container images work?
* How can I annotate a graph?

In [8]:
# Q4: Filtered search
query_q4 = "How do copy a file to a Docker container?"
course_filter_q4 = "machine-learning-zoomcamp"
third_question_q4_answer = None

search_query_q4 = {
    "size": 3, 
    "query": {
        "bool": {
            "must": {
                "multi_match": {
                    "query": query_q4,
                    "fields": ["question", "text"],
                    "type": "best_fields"
                }
            },
            "filter": {
                "term": {
                    "course": course_filter_q4
                }
            }
        }
    }
}

retrieved_hits_q4_for_q5_context = [] # Store for Q5
try:
    response_q4 = es_client.search(index=index_name, body=search_query_q4)
    retrieved_hits_q4_for_q5_context = response_q4['hits']['hits'] # Save for Q5
    
    if len(retrieved_hits_q4_for_q5_context) >= 3:
        third_question_q4_answer = retrieved_hits_q4_for_q5_context[2]['_source']['question']
        print(f"Q4 Answer: The 3rd question returned is: '{third_question_q4_answer}'")
    elif len(retrieved_hits_q4_for_q5_context) > 0:
        print(f"Only {len(retrieved_hits_q4_for_q5_context)} results returned for Q4. Cannot get the 3rd question.")
    else:
        print("No results returned for the query in Q4.")

except ConnectionError:
    print("Elasticsearch connection error for Q4. Cannot perform search.")
except Exception as e:
    print(f"An error occurred during search Q4: {e}")

Q4 Answer: The 3rd question returned is: 'How do I debug a docker container?'


In [9]:
len(retrieved_hits_q4_for_q5_context)

3

In [10]:
retrieved_hits_q4_for_q5_context[0]

{'_index': 'course-faq-homework-2025',
 '_id': 'l317IpcBSJ5i86OCcivc',
 '_score': 22.931826,
 '_source': {'text': "You can copy files from your local machine into a Docker container using the docker cp command. Here's how to do it:\nTo copy a file or directory from your local machine into a running Docker container, you can use the `docker cp command`. The basic syntax is as follows:\ndocker cp /path/to/local/file_or_directory container_id:/path/in/container\nHrithik Kumar Advani",
  'section': '5. Deploying Machine Learning Models',
  'question': 'How do I copy files from my local machine to docker container?',
  'course': 'machine-learning-zoomcamp'}}

In [11]:
retrieved_hits_q4_for_q5_context

[{'_index': 'course-faq-homework-2025',
  '_id': 'l317IpcBSJ5i86OCcivc',
  '_score': 22.931826,
  '_source': {'text': "You can copy files from your local machine into a Docker container using the docker cp command. Here's how to do it:\nTo copy a file or directory from your local machine into a running Docker container, you can use the `docker cp command`. The basic syntax is as follows:\ndocker cp /path/to/local/file_or_directory container_id:/path/in/container\nHrithik Kumar Advani",
   'section': '5. Deploying Machine Learning Models',
   'question': 'How do I copy files from my local machine to docker container?',
   'course': 'machine-learning-zoomcamp'}},
 {'_index': 'course-faq-homework-2025',
  '_id': 'mH17IpcBSJ5i86OCcivd',
  '_score': 19.161318,
  '_source': {'text': 'You can copy files from your local machine into a Docker container using the docker cp command. Here\'s how to do it:\nIn the Dockerfile, you can provide the folder containing the files that you want to copy ove

In [12]:
retrieved_hits_q4_for_q5_context[0]["_source"]["text"]

"You can copy files from your local machine into a Docker container using the docker cp command. Here's how to do it:\nTo copy a file or directory from your local machine into a running Docker container, you can use the `docker cp command`. The basic syntax is as follows:\ndocker cp /path/to/local/file_or_directory container_id:/path/in/container\nHrithik Kumar Advani"

## Q5: Building a Prompt

**Goal:** Use records from Q4 to build context. Use this context and question "How do I execute a command in a running docker container?" to construct a prompt. Find the prompt's length.

Prompt Template for Context Entries:
```
Q: {question}
A: {text}
```
Main Prompt Template:
```
You're a course teaching assistant. Answer the QUESTION based on the CONTEXT from the FAQ database.
Use only the facts from the CONTEXT when answering the QUESTION.

QUESTION: {question}

CONTEXT:
{context}
```
Options for length:
* 946
* 1446
* 1946
* 2446

In [13]:
# Q5: Building a prompt and calculating its length
final_prompt_q5_text = None
prompt_length_q5_answer = None

context_docs_q5 = []
if 'retrieved_hits_q4_for_q5_context' in locals() and retrieved_hits_q4_for_q5_context:
    for hit in retrieved_hits_q4_for_q5_context:
        context_docs_q5.append(hit['_source'])
else:
    print("Warning for Q5: Context from Q4 not available. Retrieving again.")
    # This is a fallback, ideally Q4 should populate retrieved_hits_q4_for_q5_context
    try:
        response_q4_fallback = es_client.search(index=index_name, body=search_query_q4) # Uses search_query_q4
        retrieved_hits_q4_for_q5_context = response_q4_fallback['hits']['hits']
        for hit in retrieved_hits_q4_for_q5_context:
            context_docs_q5.append(hit['_source'])
    except Exception as e:
        print(f"Error in Q5 fallback context retrieval: {e}")

context_string_q5 = ""
if context_docs_q5:
    for doc in context_docs_q5:
        context_entry = f"Q: {doc.get('question', '')}\nA: {doc.get('text', '')}"
        context_string_q5 += context_entry.strip() + "\n\n"
context_string_q5 = context_string_q5.strip()
# print("Context for Q5:\n", context_string_q5) # For debugging

question_for_prompt_q5 = "How do I execute a command in a running docker container?"
prompt_template_q5 = """
You're a course teaching assistant. Answer the QUESTION based on the CONTEXT from the FAQ database.
Use only the facts from the CONTEXT when answering the QUESTION.

QUESTION: {question}

CONTEXT:
{context}
""".strip()

if context_string_q5: # Only build prompt if context is available
    final_prompt_q5_text = prompt_template_q5.format(question=question_for_prompt_q5, context=context_string_q5)
    # print("\nFinal Prompt for Q5:\n", final_prompt_q5_text) # For debugging
    prompt_length_q5_answer = len(final_prompt_q5_text)
    print(f"Q5 Answer: The length of the resulting prompt is: {prompt_length_q5_answer}")
else:
    print("Could not build prompt for Q5 as context is empty.")

Q5 Answer: The length of the resulting prompt is: 1462


In [14]:
final_prompt_q5_text

'You\'re a course teaching assistant. Answer the QUESTION based on the CONTEXT from the FAQ database.\nUse only the facts from the CONTEXT when answering the QUESTION.\n\nQUESTION: How do I execute a command in a running docker container?\n\nCONTEXT:\nQ: How do I copy files from my local machine to docker container?\nA: You can copy files from your local machine into a Docker container using the docker cp command. Here\'s how to do it:\nTo copy a file or directory from your local machine into a running Docker container, you can use the `docker cp command`. The basic syntax is as follows:\ndocker cp /path/to/local/file_or_directory container_id:/path/in/container\nHrithik Kumar Advani\n\nQ: How do I copy files from a different folder into docker container’s working directory?\nA: You can copy files from your local machine into a Docker container using the docker cp command. Here\'s how to do it:\nIn the Dockerfile, you can provide the folder containing the files that you want to copy ov

## Q6: Token Counting

**Goal:** Calculate the number of tokens in the Q5 prompt using `tiktoken`.

Options:
* 120
* 220
* 320
* 420

In [15]:
# Q6: Token counting
num_tokens_q6_answer = None

if final_prompt_q5_text:
    try:
        encoding = tiktoken.encoding_for_model("gpt-4o") # As per homework context
        num_tokens_q6_answer = len(encoding.encode(final_prompt_q5_text))
        print(f"Q6 Answer: The number of tokens in the prompt from Q5 is: {num_tokens_q6_answer}")
        
        # Example of decoding a token (not part of the answer, just for illustration)
        # if num_tokens_q6_answer > 0:
        #     first_token = encoding.encode(final_prompt_q5_text)[0]
        #     decoded_token_bytes = encoding.decode_single_token_bytes(first_token)
        #     print(f"Example: First token ID: {first_token}, Decoded bytes: {decoded_token_bytes}")
            
    except Exception as e:
        print(f"An error occurred during token counting for Q6: {e}")
else:
    print("Prompt from Q5 (final_prompt_q5_text) is not available for Q6 token counting.")

Q6 Answer: The number of tokens in the prompt from Q5 is: 323


## Q7: OpenAI Response

**Goal:** Send the Q5 prompt to OpenAI. What's the response?

*Note: This requires a valid OpenAI API key and will incur a small cost.*

In [16]:
# Q7: Send prompt to OpenAI and get response
openai_answer_q7 = "Error: Prerequisite prompt not available or API call failed."

if final_prompt_q5_text:
    print(f"Sending prompt (length: {len(final_prompt_q5_text)} chars, approx tokens: {num_tokens_q6_answer if num_tokens_q6_answer else 'N/A'}) to OpenAI for Q7...")
    try:
        if not os.environ.get("OPENAI_API_KEY"):
            print("OPENAI_API_KEY not set. Skipping Q7 API call.")
            openai_answer_q7 = "Skipped: OPENAI_API_KEY not set."
        else:
            openai_response_q7 = client.chat.completions.create(
                model="gpt-4o", # As per homework context for pricing
                messages=[{"role": "user", "content": final_prompt_q5_text}]
            )
            openai_answer_q7 = openai_response_q7.choices[0].message.content
            print("Q7 Response from OpenAI:")
            print(openai_answer_q7)
    except NameError as e:
        print(f"Cannot proceed with Q7: {e} - ensure final_prompt_q5_text is defined.")
        openai_answer_q7 = f"Error: {e}"
    except openai.APIConnectionError as e:
        print(f"OpenAI API Connection Error for Q7: {e}")
        openai_answer_q7 = f"OpenAI API Connection Error: {e}"
    except openai.RateLimitError as e:
        print(f"OpenAI API Rate Limit Error for Q7: {e}")
        openai_answer_q7 = f"OpenAI API Rate Limit Error: {e}"
    except openai.APIStatusError as e:
        print(f"OpenAI API Status Error for Q7: {e.status_code} - {e.response}")
        openai_answer_q7 = f"OpenAI API Status Error: {e.status_code}"
    except Exception as e:
        print(f"An unexpected error occurred with OpenAI API call for Q7: {e}")
        openai_answer_q7 = f"Unexpected OpenAI API error: {e}"
else:
    print("Prompt from Q5 (final_prompt_q5_text) is not available. Skipping Q7 OpenAI call.")

Sending prompt (length: 1462 chars, approx tokens: 323) to OpenAI for Q7...
Q7 Response from OpenAI:
To execute a command in a running Docker container, use the `docker exec` command. First, you can find the container ID by using `docker ps` to list all running containers. Then, execute your desired command in the container using the following syntax:

```
docker exec -it <container-id> <command>
```

For example, to open a bash shell, you would use:

```
docker exec -it <container-id> bash
```


## Q8: Cost Calculation

**Goal:** Calculate the cost for 1000 requests with average 150 input tokens and 250 output tokens per request.
Prices for gpt-4o (June 17):
* Input: $0.005 / 1K tokens
* Output: $0.015 / 1K tokens

In [17]:
# Q8: Cost calculation

avg_input_tokens_per_request = 150
avg_output_tokens_per_request = 250
num_requests = 1000

price_input_per_1k_tokens = 0.005  # dollars
price_output_per_1k_tokens = 0.015 # dollars

total_input_tokens = avg_input_tokens_per_request * num_requests
total_output_tokens = avg_output_tokens_per_request * num_requests

cost_input = (total_input_tokens / 1000) * price_input_per_1k_tokens
cost_output = (total_output_tokens / 1000) * price_output_per_1k_tokens

total_cost_q8_answer = cost_input + cost_output

print(f"Total input tokens for {num_requests} requests: {total_input_tokens}")
print(f"Total output tokens for {num_requests} requests: {total_output_tokens}")
print(f"Cost for input tokens: ${cost_input:.2f}")
print(f"Cost for output tokens: ${cost_output:.2f}")
print(f"Q8 Answer: Total estimated cost for {num_requests} requests: ${total_cost_q8_answer:.2f}")

Total input tokens for 1000 requests: 150000
Total output tokens for 1000 requests: 250000
Cost for input tokens: $0.75
Cost for output tokens: $3.75
Q8 Answer: Total estimated cost for 1000 requests: $4.50


### Bonus: Re-calculate cost with Q6 and Q7 values (if available)

This is an optional step to apply the pricing to the actual tokens used in Q6 (prompt) and Q7 (response).

In [22]:
# Bonus: Cost calculation based on Q6 and Q7 actuals
cost_bonus = None
if num_tokens_q6_answer is not None and openai_answer_q7 and not openai_answer_q7.startswith("Error") and not openai_answer_q7.startswith("Skipped"):
    try:
        actual_input_tokens = num_tokens_q6_answer
        actual_output_tokens = len(encoding.encode(openai_answer_q7)) # Assuming 'encoding' is still defined from Q6
        
        cost_input_actual = (actual_input_tokens / 1000) * price_input_per_1k_tokens
        cost_output_actual = (actual_output_tokens / 1000) * price_output_per_1k_tokens
        cost_bonus = cost_input_actual + cost_output_actual

        print(f"\nPRICES AS OF JUNE 2024):")
        print(f"\nBonus Calculation (based on Q6 prompt and Q7 response):")
        print(f"  Actual input tokens (Q6 prompt): {actual_input_tokens}")
        print(f"  Actual output tokens (Q7 response): {actual_output_tokens}")
        print(f"  Cost for actual input: ${cost_input_actual:.6f}")
        print(f"  Cost for actual output: ${cost_output_actual:.6f}")
        print(f"  Total cost for the single Q6/Q7 RAG call: ${cost_bonus:.6f}")
    except Exception as e:
        print(f"Error in bonus cost calculation: {e}")
else:
    print("\nBonus Calculation: Skipped, as actual token counts from Q6/Q7 are not available or Q7 failed.")


PRICES AS OF JUNE 2024):

Bonus Calculation (based on Q6 prompt and Q7 response):
  Actual input tokens (Q6 prompt): 323
  Actual output tokens (Q7 response): 93
  Cost for actual input: $0.001615
  Cost for actual output: $0.001395
  Total cost for the single Q6/Q7 RAG call: $0.003010


as june 2024
Bonus Calculation (based on Q6 prompt and Q7 response):
  Actual input tokens (Q6 prompt): 323
  Actual output tokens (Q7 response): 85
  Cost for actual input: $0.001615
  Cost for actual output: $0.001275
  Total cost for the single Q6/Q7 RAG call: $0.002890

as may 2025 con 4o


gpt-4o
gpt-4o-2024-08-06
	
$2.50
	
$1.25
	
$10.00

In [24]:
price_input_per_1k_tokens_4o = 2.5/1000  # dollars
price_output_per_1k_tokens_4o = 10/1000 # dollars

In [25]:
# Bonus: Cost calculation based on Q6 and Q7 actuals
cost_bonus = None
if num_tokens_q6_answer is not None and openai_answer_q7 and not openai_answer_q7.startswith("Error") and not openai_answer_q7.startswith("Skipped"):
    try:
        actual_input_tokens = num_tokens_q6_answer
        actual_output_tokens = len(encoding.encode(openai_answer_q7)) # Assuming 'encoding' is still defined from Q6
        
        cost_input_actual = (actual_input_tokens / 1000) * price_input_per_1k_tokens_4o
        cost_output_actual = (actual_output_tokens / 1000) * price_output_per_1k_tokens_4o
        cost_bonus = cost_input_actual + cost_output_actual

        print(f"\nPRICES AS OF MAY 2025):")
        
        print(f"\nBonus Calculation (based on Q6 prompt and Q7 response):")
        print(f"  Actual input tokens (Q6 prompt): {actual_input_tokens}")
        print(f"  Actual output tokens (Q7 response): {actual_output_tokens}")
        print(f"  Cost for actual input: ${cost_input_actual:.9f}")
        print(f"  Cost for actual output: ${cost_output_actual:.9f}")
        print(f"  Total cost for the single Q6/Q7 RAG call: ${cost_bonus:.9f}")
    except Exception as e:
        print(f"Error in bonus cost calculation: {e}")
else:
    print("\nBonus Calculation: Skipped, as actual token counts from Q6/Q7 are not available or Q7 failed.")


PRICES AS OF MAY 2025):

Bonus Calculation (based on Q6 prompt and Q7 response):
  Actual input tokens (Q6 prompt): 323
  Actual output tokens (Q7 response): 93
  Cost for actual input: $0.000807500
  Cost for actual output: $0.000930000
  Total cost for the single Q6/Q7 RAG call: $0.001737500
